## Evaluate candidate models with BERTScore for contextual similarity to ground truth answer

##### Prerequisite 

In [ ]:
%%capture

!pip install transformers==4.18.0
!pip install pandas==1.4.1
!pip install numpy==1.22.2
!pip install torch==1.8.1
!pip install evaluate==0.4.0
!pip install bert-score==0.3.12

#### Imports 

In [2]:
from transformers import GPT2Tokenizer
from transformers import set_seed
from evaluate import load
import transformers 
import pandas as pd
import numpy as np
import bert_score
import evaluate
import logging
import torch

##### Setup logging 

In [3]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

##### Log versions of dependencies 

In [4]:
logger.info(f'[Using transformers version: {transformers.__version__}]')
logger.info(f'[Using bert_score version: {bert_score.__version__}]')
logger.info(f'[Using evaluate version: {evaluate.__version__}]')
logger.info(f'[Using torch version: {torch.__version__}]')
logger.info(f'[Using pandas version: {pd.__version__}]')
logger.info(f'[Using numpy version: {np.__version__}]')

[Using transformers version: 4.18.0]
[Using bert_score version: 0.3.12]
[Using evaluate version: 0.4.0]
[Using torch version: 1.8.1+cu102]
[Using pandas version: 1.4.1]
[Using numpy version: 1.22.2]


#### Setup essentials 

In [5]:
set_seed(123)
np.random.seed(123)
pd.options.display.max_colwidth = None

In [6]:
BOS_TOKEN = '<|startoftext|>'
EOS_TOKEN = '<|endoftext|>'
PAD_TOKEN = '<|pad|>'
MAX_LEN = 512

In [7]:
bertscore = load('bertscore')

#### Load custom tokenizer 

In [8]:
custom_tokenizer = GPT2Tokenizer.from_pretrained('../01-tokenize/vocab-custom', 
                                                 bos_token=BOS_TOKEN, 
                                                 eos_token=EOS_TOKEN, 
                                                 pad_token=PAD_TOKEN, 
                                                 lower=True,
                                                 return_tensors='pt')
custom_tokenizer.padding_side = 'left'
custom_tokenizer.model_max_length = MAX_LEN
logger.info(f'Custom Tokenizer: {custom_tokenizer}')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Custom Tokenizer: PreTrainedTokenizer(name_or_path='../01-tokenize/vocab-custom', vocab_size=50257, model_max_len=512, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'bos_token': AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<|pad|>'})


#### Load OOB tokenizer 

In [9]:
oob_tokenizer = GPT2Tokenizer.from_pretrained('gpt2', 
                                              bos_token=BOS_TOKEN, 
                                              eos_token=EOS_TOKEN, 
                                              pad_token=PAD_TOKEN, 
                                              lower=True,
                                              return_tensors='pt')
oob_tokenizer.padding_side = 'left'
oob_tokenizer.model_max_length = MAX_LEN
logger.info(f'OOB Tokenizer: {oob_tokenizer}')

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
OOB Tokenizer: PreTrainedTokenizer(name_or_path='gpt2', vocab_size=50257, model_max_len=512, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'bos_token': AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<|pad|>'})


#### Load custom GPT2 model

In [10]:
custom_model = transformers.AutoModelForCausalLM.from_pretrained('.././02-finetune/model/custom-finetuned')
_ = custom_model.eval()

#### Load OOB GPT2 model

In [11]:
oob_model = transformers.AutoModelForCausalLM.from_pretrained('.././02-finetune/model/oob-finetuned')
_ = oob_model.eval()

#### Load test set 

In [12]:
test_df = pd.read_csv('.././01-tokenize/data/faq_test.csv')
test_df.count()

question    107
answer      107
dtype: int64

#### Collect predicted responses

In [13]:
def predict(question: str, ground_truth: str, tokenizer: GPT2Tokenizer, model: transformers.AutoModelForCausalLM) -> str:
    # create a prompt in compliance with the one used during training without the answer part
    prompt = f'{BOS_TOKEN}question: {question}\nanswer:'
    # generate tokens
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids
    # predict response (answer)
    gt_len = len(ground_truth.split())
    response = model.generate(input_ids, 
                              do_sample=True, 
                              top_k=1, 
                              min_new_tokens=gt_len * 2,
                              max_new_tokens=gt_len * 2, 
                              repetition_penalty=10.0,
                              length_penalty=-0.1,
                              top_p=1.0)
    # decode the predicted tokens into texts
    response_text = tokenizer.decode(response[0], skip_special_tokens=True)
    answer = response_text.split('answer: ')[-1]
    return answer

In [14]:
custom_gpt2_answers = []
oob_gpt2_answers = []

for _, row in test_df.iterrows():
    question, ground_truth = row
    answer = predict(question, ground_truth, custom_tokenizer, custom_model)
    custom_gpt2_answers.append(answer)
    answer = predict(question, ground_truth, oob_tokenizer, oob_model)
    oob_gpt2_answers.append(answer)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_i

#### Compute BERTScore for the predictions against ground truth

In [15]:
bert_score_custom_gpt2 = bertscore.compute(predictions=custom_gpt2_answers, references=test_df['answer'].to_list(), lang='en')['f1']
bert_score_oob_gpt2 = bertscore.compute(predictions=oob_gpt2_answers, references=test_df['answer'].to_list(), lang='en')['f1']
    
test_df['custom_gpt2_answer'] = custom_gpt2_answers
test_df['oob_gpt2_answer'] = oob_gpt2_answers

test_df['bert_score_custom_gpt2'] = bert_score_custom_gpt2
test_df['bert_score_oob_gpt2'] = bert_score_oob_gpt2

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

In [16]:
test_df.head()

,question,answer,custom_gpt2_answer,oob_gpt2_answer,bert_score_custom_gpt2,bert_score_oob_gpt2
0,"i have a few symptoms like the stomachache, congestion, and diarrhea but no fever. is it possible i have the virus?","stomach troubles aren't a common symptom of the coronavirus, but a fever is a key symptom, so it's unlikely that you have it. however, if you have any questions, call ahead to your doctor and make an appointment.",yes! while we are still learning about how covid-2019 affects people’s health more information can be found on cdc website (pdf),yes! you may be able to get covid-19 from eating raw meat or poultry that has been contaminated with sarsaparic acid (sARS) in your food source for at least 20 days after exposure so there are some signs of illness associated if this happens during those 30 day period when people eat foods high on antibiotics such as chicken breast soup instead,0.830807,0.819558
1,what if my time off is not approved and i don’t come to work?,you will be treated just as you would if you did not show up for work any other time. employees failing to come to work without approved leave time may be subject to discipline per the applicable collective bargaining agreement or compensation plan.,"you can volunteer at a food bank or other community organization. make sure they have the supplies where their workers are, as well your insurance information so that when someone in an essential business like grocery stores becomes ill it doesn't affect them financially but cannot donate money directly toward those who need help with groceries during this pandemic (or else)","you can contact your employer or the local fire department for assistance with contacting their office regarding a potential employee who may be unable due leave. they will provide information about how long it takes them until an individual has been in place, such as when hours are reduced (e-mailing) onsite/out of state by telephone call; where employees should go out during this period without interruption while working from home but do so at least",0.817220,0.832336
2,where can i find more information about animals and covid-19?,"for more information, check out the following websites: cdc covid-19: pets and other animals cdc: frequently asked questions, covid-19 and animals cdc: interim guidance for public health professionals managing people with covid-19 in home care and isolation who have pets or other animals cdc: healthy pets, healthy people cdc: covid-19 daily life and coping, if you have animals avma: interim recommendations for intake of companion animals from households where humans with covid-19 are present","the centers for disease control (cdc) is constantly updating its website with resources to help people understand this new virus. if you or someone in your family has questions, reach out via email at afnrsa@vdh3sorg/coronavirus—or call 804‑234–7364 –and ask them on facebook messenger! they’ll answer any queries that come their way through these chats; otherwise we encourage anyone who's interested by reading what cdc guidance applies here.","the cdc has a list of animal health organizations that are working on prevention, treatment strategies for people with or without symptoms associatedcov infection. see https://www/cdphs’vegetarianscoronavirus/.",0.798086,0.806660
3,what precautions should i take during travel?,"during travel, everyone should clean hands frequently, cough or sneeze into a bent elbow or tissue, and try to maintain a physical distance of at least one metre from others. travelers should follow the recommendations of the travel authorities regarding policies in the airport and of the airline for the flight.","if you have been in close contact with someone suspected of having covid-19, or are feeling unwell (like coughing), avoid touching your face. this includes phones and other electronic devices that might be used by people who were not well before the outbreak began—something like shaking hands; washi

In [17]:
np.mean(test_df['bert_score_custom_gpt2'])

0.8332844811065174

In [18]:
np.mean(test_df['bert_score_oob_gpt2'])

0.8321203411182511

#### Write evaluation results to local dir

In [19]:
test_df.to_csv('./data/eval_results.csv', index=False)

#### Create classification dataset for BERT finetuning

In [27]:
results_df = pd.read_csv('./data/eval_results.csv')
results_df.head()

,question,answer,custom_gpt2_answer,oob_gpt2_answer,bert_score_custom_gpt2,bert_score_oob_gpt2
0,"i have a few symptoms like the stomachache, congestion, and diarrhea but no fever. is it possible i have the virus?","stomach troubles aren't a common symptom of the coronavirus, but a fever is a key symptom, so it's unlikely that you have it. however, if you have any questions, call ahead to your doctor and make an appointment.",yes! while we are still learning about how covid-2019 affects people’s health more information can be found on cdc website (pdf),yes! you may be able to get covid-19 from eating raw meat or poultry that has been contaminated with sarsaparic acid (sARS) in your food source for at least 20 days after exposure so there are some signs of illness associated if this happens during those 30 day period when people eat foods high on antibiotics such as chicken breast soup instead,0.830807,0.819558
1,what if my time off is not approved and i don’t come to work?,you will be treated just as you would if you did not show up for work any other time. employees failing to come to work without approved leave time may be subject to discipline per the applicable collective bargaining agreement or compensation plan.,"you can volunteer at a food bank or other community organization. make sure they have the supplies where their workers are, as well your insurance information so that when someone in an essential business like grocery stores becomes ill it doesn't affect them financially but cannot donate money directly toward those who need help with groceries during this pandemic (or else)","you can contact your employer or the local fire department for assistance with contacting their office regarding a potential employee who may be unable due leave. they will provide information about how long it takes them until an individual has been in place, such as when hours are reduced (e-mailing) onsite/out of state by telephone call; where employees should go out during this period without interruption while working from home but do so at least",0.817220,0.832336
2,where can i find more information about animals and covid-19?,"for more information, check out the following websites: cdc covid-19: pets and other animals cdc: frequently asked questions, covid-19 and animals cdc: interim guidance for public health professionals managing people with covid-19 in home care and isolation who have pets or other animals cdc: healthy pets, healthy people cdc: covid-19 daily life and coping, if you have animals avma: interim recommendations for intake of companion animals from households where humans with covid-19 are present","the centers for disease control (cdc) is constantly updating its website with resources to help people understand this new virus. if you or someone in your family has questions, reach out via email at afnrsa@vdh3sorg/coronavirus—or call 804‑234–7364 –and ask them on facebook messenger! they’ll answer any queries that come their way through these chats; otherwise we encourage anyone who's interested by reading what cdc guidance applies here.","the cdc has a list of animal health organizations that are working on prevention, treatment strategies for people with or without symptoms associatedcov infection. see https://www/cdphs’vegetarianscoronavirus/.",0.798086,0.806660
3,what precautions should i take during travel?,"during travel, everyone should clean hands frequently, cough or sneeze into a bent elbow or tissue, and try to maintain a physical distance of at least one metre from others. travelers should follow the recommendations of the travel authorities regarding policies in the airport and of the airline for the flight.","if you have been in close contact with someone suspected of having covid-19, or are feeling unwell (like coughing), avoid touching your face. this includes phones and other electronic devices that might be used by people who were not well before the outbreak began—something like shaking hands; washi

In [28]:
dataset = {}

In [29]:
for _, row in results_df.iterrows():
    _, _, custom_gpt2_answer, oob_gpt2_answer, bert_score_custom_gpt2, bert_score_oob_gpt2 = row
    if bert_score_custom_gpt2 > bert_score_oob_gpt2:
        dataset[custom_gpt2_answer] = 1
        dataset[oob_gpt2_answer] = 0
    elif bert_score_custom_gpt2 < bert_score_oob_gpt2:
        dataset[custom_gpt2_answer] = 0
        dataset[oob_gpt2_answer] = 1
    else:  # corner case when the scores are equal 
        len_1 = len(custom_gpt2_answer)
        len_2 = len(oob_gpt2_answer)
        if len_1 < len_2:
            dataset[custom_gpt2_answer] = 1
            dataset[oob_gpt2_answer] = 0
        elif len_1 > len_2:
            dataset[custom_gpt2_answer] = 0
            dataset[oob_gpt2_answer] = 1
        else:
            # extreme corner case when both responses are either identical or of the same length with the same BERTscores 
            dataset[custom_gpt2_answer] = 1
            dataset[oob_gpt2_answer] = 0

In [32]:
clf_dataset = pd.DataFrame(list(dataset.items()), columns=['response', 'label'])
clf_dataset.head()

,response,label
0,yes! while we are still learning about how covid-2019 affects people’s health more information can be found on cdc website (pdf),1
1,yes! you may be able to get covid-19 from eating raw meat or poultry that has been contaminated with sarsaparic acid (sARS) in your food source for at least 20 days after exposure so there are some signs of illness associated if this happens during those 30 day period when people eat foods high on antibiotics such as chicken breast soup instead,0
2,"you can volunteer at a food bank or other community organization. make sure they have the supplies where their workers are, as well your insurance information so that when someone in an essential business like grocery stores becomes ill it doesn't affect them financially but cannot donate money directly toward those who need help with groceries during this pandemic (or else)",0
3,"you can contact your employer or the local fire department for assistance with contacting their office regarding a potential employee who may be unable due leave. they will provide information about how long it takes them until an individual has been in place, such as when hours are reduced (e-mailing) onsite/out of state by telephone call; where employees should go out during this period without interruption while working from home but do so at least",1
4,"the centers for disease control (cdc) is constantly updating its website with resources to help people understand this new virus. if you or someone in your family has questions, reach out via email at afnrsa@vdh3sorg/coronavirus—or call 804‑234–7364 –and ask them on facebook messenger! they’ll answer any queries that come their way through these chats; otherwise we encourage anyone who's interested by reading what cdc guidance applies here.",0


In [33]:
clf_dataset.count()

response    214
label       214
dtype: int64

##### Write the clf dataset to local dir

In [34]:
clf_dataset.to_csv('./data/clf_dataset.csv', index=False)